# Import RPB model along with other utility functions

In [1]:
from RPB_model import *

expression symbols from pyomo.core.expr  (deprecated in 6.6.2.dev0) (called
from <frozen importlib._bootstrap>:241)


# Creating model for a single side/section of the RPB

In [2]:
# Create model instance. Currently, mode can be either "adsorption" or "desorption" which sets the boundary conditions for each case.
# m=RPB_model(mode="adsorption")
# or
m=RPB_model(mode="adsorption", gas_flow_direction=1)

# Custom initialization routine. Uses a homotopy routine to adjust the initialization factors so that all equations are 'active'.
single_section_init(m)
# homotopy_init_routine(m)

DOF = 0
Ipopt 3.13.2: 

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt

This version of Ipopt was compiled from source code available at
    https://github.com/IDAES/Ipopt as part of the Institute for the Design of
    Advanced Energy Systems Process Systems Engineering Framework (IDAES PSE
    Framework) Copyright (c) 2018-2019. See https://github.com/IDAES/idaes-pse.

This version of Ipopt was compiled using HSL, a collection of Fortran codes
    for large-scale scientific computation.  All technical papers, sales and
    publicity material resulting from use of the HSL codes within IPOPT must
    contain the following acknowledgement:
        HSL, a collection of Fortran codes for large-scale scientific
        computation. See

In [3]:
# Some various utility functions that I have been using to check model performance
evaluate_MB_error(m)

print(f'CO2 Capture = {m.CO2_capture():.3}')

check_scaling(m)

# degen_hunter(m)

jac, variables, constraints = scaling_script(m)

N2 error = -3.67e-10 %
CO2 error = 1.11 %
H2O error = -1.62e-09 %
CO2 Capture = 0.853
component keys that are not exported as part of the NL file.  Skipping.
Jacobian Condition Number: 9.01e+06
component keys that are not exported as part of the NL file.  Skipping.
Badly scaled variables:


Badly scaled constraints:

Spectral condition number: 1.900e+05

Smallest singular value: 0.004645519059879676

Variables in smallest singular vector:
1401: Flux_kzo[CO2,1,0]
1405: Flux_kzo[CO2,1,1]
1407: Flux_kzo[H2O,1,0]
1411: Flux_kzo[H2O,1,1]
2165: Flux_kzo[H2O,0.275,0]
2169: Flux_kzo[H2O,0.275,1]
2183: Flux_kzo[H2O,0.3875,0]
2187: Flux_kzo[H2O,0.3875,1]
2201: Flux_kzo[H2O,0.5,0]
2205: Flux_kzo[H2O,0.5,1]
2213: Flux_kzo[CO2,0.725,0]
2217: Flux_kzo[CO2,0.725,1]
2219: Flux_kzo[H2O,0.725,0]
2223: Flux_kzo[H2O,0.725,1]
2231: Flux_kzo[CO2,0.95,0]
2235: Flux_kzo[CO2,0.95,1]
2237: Flux_kzo[H2O,0.95,0]
2241: Flux_kzo[H2O,0.95,1]
2249: Flux_kzo[CO2,0.96,0]
2253: Flux_kzo[CO2,0.96,1]
2255: Flux_kzo[H2O,0.

In [ ]:
print(jac.getcol(3076))

In [ ]:
variables[50].pprint()

In [ ]:
constraints[2776].pprint()

In [ ]:
constraints[3835].pprint()

In [ ]:
constraints[3836].pprint()

In [ ]:
constraints[2496].pprint()

In [ ]:
variables[461].pprint()

In [ ]:
variables[768].pprint()

In [ ]:
[m.Rs_CO2[z,o]() for z,o in m.dheat_fluxdz.keys()]

In [ ]:
print(iscale.get_scaling_factor(constraints[927]))

# Creating a full RPB model

In [ ]:
RPB = full_model_creation(lean_temp_connection=True, configuration = "counter-current")
init_routine_1(RPB)

In [ ]:
fix_capture_and_solve(RPB, capture=0.95)

In [ ]:
report(RPB)

In [ ]:
evaluate_MB_error(RPB.ads)

In [ ]:
check_scaling(RPB)

In [ ]:
jac, variables, constraints = scaling_script(RPB)

In [ ]:
print(jac.getrow(4381))

In [ ]:
constraints[2722].pprint()

In [ ]:
constraints[3964].pprint()

In [ ]:
variables[1760].pprint()

In [ ]:
variables[1739].pprint()

In [ ]:
variables[7419].pprint()

In [ ]:
print(iscale.get_scaling_factor(constraints[2722]))

In [ ]:
print(iscale.get_scaling_factor(variables[4336]))

In [ ]:
[RPB.ads.dTsdo[z,o]() for z,o in RPB.ads.dTsdo.keys()]

# Save and load model

In [ ]:
# save model
to_json(RPB, fname="counter current low rpm case.json.gz", gz=True, human_read=False)

In [ ]:
# create model, load, and solve
RPB = full_model_creation(lean_temp_connection=True, configuration = "counter-current")

from_json(RPB, fname="counter current base case 090823.json.gz", gz=True)

solver = SolverFactory("ipopt")
solver.options = {
    "max_iter": 1000,
    "bound_push": 1e-22,
    # "halt_on_ampl_error": "yes",
}
solver.solve(RPB, tee=True).write()

Optimization Testing

In [ ]:
# create regularization parameter for the objective function
RPB.alpha_obj = Param(initialize=0.9, mutable=True)

# add objective
@RPB.Expression()
def obj(RPB):
    return RPB.alpha_obj * RPB.energy_requirement/10 - (1 - RPB.alpha_obj) * RPB.productivity/20

RPB.objective = Objective(expr=RPB.obj)

In [ ]:
RPB.ads.L.setlb(0.5)
RPB.ads.L.setub(10)
RPB.ads.L.pprint()

In [ ]:
RPB.ads.Tx.setlb(25+273)
RPB.ads.Tx.setub(95+273)
RPB.ads.Tx.pprint()

In [ ]:
RPB.des.Tx.setlb(100+273)
RPB.des.Tx.setub(160+273)
RPB.des.Tx.pprint()

In [ ]:
RPB.des.P_in.setub(1.15)
RPB.des.P_in.setlb(1.0)
RPB.des.P_in.pprint()

In [ ]:
degrees_of_freedom(RPB)

In [ ]:
RPB.ads.w_rpm.setlb(0.5)
RPB.ads.w_rpm.setub(5)
RPB.ads.w_rpm.pprint()

In [ ]:
# free up decision variables
# RPB.ads.L.unfix()
# RPB.ads.theta.unfix()
RPB.des.P_in.unfix()
# RPB.ads.Tx.unfix()
# RPB.des.Tx.unfix()

In [ ]:
degrees_of_freedom(RPB)

In [ ]:
# solve
solver = SolverFactory("ipopt")
solver.options = {
    "max_iter": 1000,
    "bound_push": 1e-8,
    # "halt_on_ampl_error": "yes",
    "tol": 1e-4,
    "max_cpu_time": 5*60,
    # "mu_strategy": "adaptive",
}
solver.solve(RPB, tee=True).write()

In [ ]:
report(RPB)

In [ ]:
[RPB.ads.qCO2_eq[z,o]() for z,o in RPB.ads.qCO2_eq.keys()]

In [ ]:
RPB.ads.Tx()

In [ ]:
full_contactor_plotting(RPB)

In [ ]:
RPB.ads.Q_ghx_tot_kW()

In [ ]:
len(list(RPB.ads.z))